### Importar base de dados e instalar framework de recomendação

### Importar bibliotecas

In [ ]:
! wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
! tar -xvzf ml-20m-compact.tar.gz
! pip install caserecommender

In [ ]:
import pandas as pd
import numpy as np

### Explorar Dados

In [ ]:
movies = pd.read_csv('./dataset/movies_sample.csv')
movies.tail()

In [ ]:
ratings = pd.read_csv('./dataset/ratings_sample.csv')
ratings.head()

In [ ]:
ratings.rating.value_counts().plot(kind='bar', color=['r', 'g', 'y', 'c', 'b']);

In [ ]:
df = ratings[['userId', 'movieId', 'rating']]
df.tail()

In [ ]:
df = df.merge(movies[['movieId', 'title']])
df.head()

### Números de usuários e número de itens

In [ ]:
print(
"""
Número de usuários: {}
Número de itens: {}
Número de interações: {}
""".format(
    df.userId.nunique(),
    df.movieId.nunique(),
    df.shape[0]
)
)

### Mapeamento em idx

In [ ]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

In [ ]:
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
df.head()

In [ ]:
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title

In [ ]:
print(len(map_title))
print(map_title[100])

In [ ]:
# qtd interações dos usuários
df.groupby('userId').count()

### Divisão do dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df, test_size=.2, random_state=2)

In [ ]:
train.to_csv('train.txt', index=False, header=False, sep='\t')
test.to_csv('test.txt', index=False, header=False, sep='\t')

In [ ]:
! ls -l

In [ ]:
test.shape[0], train.shape[0]

### Recomendadores

## Prever notas

In [ ]:
from caserec.recommenders.rating_prediction.most_popular import MostPopular

In [ ]:
MostPopular('train.txt', 'test.txt', 'out_mp_pred.txt').compute()

In [ ]:
df_pred = pd.read_csv('out_mp_pred.txt', sep='\t', names=['userId', 'movieId', 'pred'])
df_pred.head()

In [ ]:
test[test.userId == 0]

In [ ]:
df_pred = df_pred.merge(test)

In [ ]:
df_pred.rating.value_counts().plot(kind='bar')

In [ ]:
df_pred['pred'] = round(df_pred['pred']*2)/2
df_pred.pred.value_counts().plot(kind='bar');

### Top N

In [ ]:
from caserec.recommenders.item_recommendation.most_popular import MostPopular as MPR

In [ ]:
MPR('train.txt', 'test.txt', 'out_mp_pred.txt').compute()

In [ ]:
MPR('train.txt', 'test.txt', 'out_mp_pred_binary.txt', as_binary=True).compute()

In [ ]:
ranking = pd.read_csv('out_mp_pred.txt', sep='\t', names=['user_id', 'movieId', 'score'])
ranking['title'] = ranking.movieId.map(map_title)
ranking.head(15)

In [ ]:
ranking = pd.read_csv('out_mp_pred_binary.txt', sep='\t', names=['userId', 'movieId', 'score'])
ranking['title'] = ranking.movieId.map(map_title)
ranking.head(15)